In [ ]:
import numpy as np

In [4]:
class Gaussian_total:
    def __init__(self,Gaussian_mean =[],Gaussian_var =[],Gaussian_weight =[],number_of_Gaussian = 0):
        self.Gaussian_mean =[]
        self.Gaussian_var =[]
        self.Gaussian_weight =[]
        self.number_of_Gaussian = number_of_Gaussian
class HMM_total:
    def __init__(self, initial_matrix=[], transition_cost =[], state_mix =[],state_number = 0 ):
        self.initial_matrix=initial_matrix
        self.transition_cost=transition_cost
        self.state_mix=state_mix

In [ ]:
def gaussian(data,sigma_square,ftheta):
    sigma_square_producted=np.prod(sigma_square,axis=1)
    p=(1/(np.sqrt((2*np.pi)**2 *sigma_square)))*(np.exp(-0.5*np.sum((data-ftheta)**2/sigma_square_producted,axis=1)))
    return p
def log_gaussian(data,sigma_square,ftheta):
    cost= 0.5*np.sum(np.log((2*np.pi)*(sigma_square)),axis=1)+0.5*np.sum(np.square((ftheta-data))/sigma_square,axis=1)
    return cost
def mix_log_gaussian(mix_gaussian,ftheta):
    Gaussian_mean=mix_gaussian.Gaussian_mean
    Gaussian_var=mix_gaussian.Gaussian_var
    Gaussian_weight=mix_gaussian.Gaussian_weight
    cost=log_gaussian(Gaussian_mean,Gaussian_var,ftheta)
    changed_cost=np.sum(Gaussian_weight*cost)
    return changed_cost
def mix_gaussian(mix_gaussian,ftheta):
    cost=0
    for i in range(0,mix_gaussian.number_of_Gaussian):
        Gaussian_mean=mix_gaussian.Gaussian_mean[i][:]
        sigma_square=mix_gaussian.Gaussian_var[i][:]
        Gaussian_weight=mix_gaussian.Gaussian_weight[i]
        cost0=gaussian(Gaussian_mean,sigma_square,ftheta)
        cost=cost+Gaussian_weight*cost0
    return cost

In [ ]:
def GMM_HMM_DTW(HMM_total,data):
    mark=False
    transition_cost=HMM_total.transition_cost
    temp_list0=np.zeros([39])
    temp_list1=np.zeros([39])+1
    mixture_of_all_states=[]
    new_GMM=Gaussian_total()
    new_GMM.Gaussian_mean.append(temp_list0)
    new_GMM.Gaussian_var.append(temp_list1)
    new_GMM.Gaussian_weight=[1]
    new_GMM.number_of_Gaussian = 1
    mixture_of_all_states.append(new_GMM)
    for mix in HMM_total.state_mix:
        mixture_of_all_states.append(mix)
    data=np.vstack([temp_list0,data])
    len_mix_of_all_states=len(mixture_of_all_states)
    len_data=len(data)
    P=np.zeros([len_mix_of_all_states,len_data])
    # Finding minimum-cost path
    for j in range(0,len_data):
      for i in range(len_mix_of_all_states):
            C_ij= mix_log_gaussian(mixture_of_all_states[i],data[j])
            if i-2>=0:
                P[i][j]=min(P[i][j-1]+transition_cost[i][i],P[i-1][j-1]+transition_cost[i-1][i],
                            P[i-2][j-1]+transition_cost[i-2][i])+C_ij
            elif i-1>=0:
                P[i][j]=min(P[i][j-1]+transition_cost[i][i],P[i-1][j-1]+transition_cost[i-1][i])+C_ij
            else:
                P[i][j]=P[i][j]+C_ij
    P=P/len_data
    distance=P[-1][-1]
    if mark:
        return distance,get_states(P)
    else:
        return distance

In [ ]:
def TransitionScore(states,state=5):
    score=np.full((state+1,state+1),np.inf)
    num_nodes_in_state=np.zeros(state+1)
    for i in range(len(states)):
        for j in range(len(states[i])):
            num_nodes_in_state[states[i][j]]+=1
    for i in range(1,state+1):
        for j in range(state+1):
            if i==j: score[i][j]=-np.log((num_nodes_in_state[i]-state)/num_nodes_in_state[i])
            elif i==j-1: score[i][j]=-np.log(state/num_nodes_in_state[i])
    score[0][1]=0
    score[state][state]=0
    return score

In [2]:
def get_states(P):
    current_state,current_frame=np.array(P.shape)-1
    current_state_array=[current_state]
    while True:
      if current_state<=0 or current_frame<=1:
        break
      current_frame-=1
      if current_state>2:
          to_check=[P[current_state][current_frame-1],P[current_state-1][current_frame-1],P[current_state-2][current_frame-1]]
          track=np.argmin(to_check)
      elif current_state>1:
          to_check=[P[current_state][current_frame-1],P[current_state-1][current_frame-1]]
          track=np.argmin(to_check)
      else:
          track=0
      if track==0:
          current_state_array.insert(0,current_state)
      elif track==1:
          current_state-=1
          current_state_array.insert(0,current_state)
      else:
          current_state-=2
          current_state_array.insert(0,current_state)
    return current_state_array

In [3]:
def get_node_in_each_state(data,state_number,node_state):
    state_list=[]
    for state in range(state_number+1):
        state_list.append([])
    for k in range(len(data)):
        for i in range(len(node_state[k])):
            j=int(node_state[k][i])
            state_list[j].append(data[k][i])
    return state_list